In [1]:
from dotenv import load_dotenv
load_dotenv()
from rtai.persona.agent import Agent
from rtai.persona.agent_manager import AgentManager
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from typing import List, Tuple

In [2]:
class TestAgent(Agent):
    def __init__(self, agent_mgr: AgentManager, event_queue: Queue, client: LLMClient, file_path: str=""):
        super().__init__(agent_mgr, event_queue, client, file_path)
    
    def _create_day_plan(self, new_day: bool, first_day: bool) -> None:
        wake_up_hour = self.s_mem.generate_wake_up_hour()
        daily_req = self.generate_daily_plan() # TODO
        return self.generate_hourly_schedule(wake_up_hour, daily_req) # TODO

    def generate_daily_plan(self) -> str:
        """   Generates the daily plan for the persona. 
        Basically the long term planning that spans a day. Returns a list of actions
        that the persona will take today. Usually comes in the following form: 
        'wake up and complete the morning routine at 6:00 am', 
        'eat breakfast at 7:00 am',.. 
        Note that the actions come without a period. 

        Persona state: identity stable set, lifestyle, cur_data_str, first_name

        INPUT: 
            persona: The Persona class instance 
            wake_up_hour: an integer that indicates when the hour the persona wakes up 
                        (e.g., 8)
        OUTPUT: 
            a list of daily actions in broad strokes.
        EXAMPLE OUTPUT: 
            ['wake up and complete the morning routine at 6:00 am', 
            'have breakfast and brush teeth at 6:30 am',
            'work on painting project from 8:00 am to 12:00 pm', 
            'have lunch at 12:00 pm', 
            'take a break and watch TV from 2:00 pm to 4:00 pm', 
            'work on painting project from 4:00 pm to 6:00 pm', 
            'have dinner at 6:00 pm', 'watch TV from 7:00 pm to 8:00 pm']"""
        
        PROMPT = """Based on the description of a person, create a daily plan for that person with a list of actions that the persona will take today. Output the plans in a list separated by commas.
        """

        return self.llm_client.generate_from_prompt(system_prompt=PROMPT, user_prompt=self.get_common_set_str())


    def generate_hourly_schedule(self, wake_up_hour, daily_req) -> List[Tuple]:
        """ * Uses LLM
        TODO
        Implement function to generate an hourly schedule for the day using LLMs
        Try and moving to a memory class

        OUTPUT:
        (activity, start_time, duration)


        """
        hour_str = ["00:00 AM", "01:00 AM", "02:00 AM", "03:00 AM", "04:00 AM", 
                    "05:00 AM", "06:00 AM", "07:00 AM", "08:00 AM", "09:00 AM", 
                    "10:00 AM", "11:00 AM", "12:00 PM", "01:00 PM", "02:00 PM", 
                    "03:00 PM", "04:00 PM", "05:00 PM", "06:00 PM", "07:00 PM",
                    "08:00 PM", "09:00 PM", "10:00 PM", "11:00 PM"]
        
        PROMPT = f"""
                Based on the description of a person and their high level plan for the day, create an hourly schedule for that person with a list of actions.
                Output the plans in a list separated by commas. Every plan should have an activity, start time, and duration!
                Here are the hours that you should use for scheduling:\{hour_str}
                """
        return self.llm_client.generate_from_prompt(system_prompt=PROMPT, user_prompt=self.get_common_set_str() + "\n\n Daily Plan:" + daily_req)


In [3]:
cfg = YamlLoader.load("configs/rtai.yaml")
event_queue = Queue()
llm_client = LLMClient(cfg.expand('LLMClient'))
mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client)
a = TestAgent(mgr, event_queue, llm_client, file_path='tests/samples/personas/persona2.txt')

In [8]:
wake_up_hour = a.s_mem.generate_wake_up_hour()
daily_req = a.generate_daily_plan() # TODO
sched = a.generate_hourly_schedule(wake_up_hour, daily_req) # TODO

In [9]:
print(daily_req)

- Wake up at 7:00 AM, have breakfast with Lois Lane
- Exercise for 30 minutes to maintain physical fitness
- Attend a press conference to cover a breaking news story as Clark Kent
- Write an article about the news story for the Daily Planet
- Meet with Lois to review and edit the article
- Visit the Justice League to discuss ongoing cases and coordinate efforts
- Have lunch with Jonathan and Martha Kent to catch up on family news
- Spend some time reflecting on his Kryptonian heritage and Earthly upbringing
- Conduct interviews for a profile piece on Lex Luthor's latest project
- Attend a charity event as Superman to raise awareness and funds for various causes
- Have dinner with Lois to debrief about the day's events
- Return home to rest and recharge for the next day


In [12]:
print(sched.split(', '))

['"Wake up at 7:00 AM', 'have breakfast with Lois Lane', 'exercise for 30 minutes', 'attend press conference', 'write article', 'meet with Lois to review', 'visit Justice League', 'lunch with family', 'reflect on heritage', 'conduct interviews', 'attend charity event', 'dinner with Lois', 'rest."']
